<h1>Chapter 12. Special Methods for Sequences</h1>

<h2><code>Vector</code> Attempt No.1: Compatibility with <code>Vector2d</code></h2>

Implementation of the first version of the `Vector` class (based on the code of the `Vector2d` class from chapter 11).

In [1]:
import math
import reprlib
from array import array


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        # Use reprlib.repr to get a limited length representation
        components = reprlib.repr(self._components)
        # Remove the prefix "array('d'" and closing bracket
        components = components[components.find('['): -1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes(ord[self.typecode]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)

In [2]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [3]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [4]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

<h2><code>Vector</code> Attempt No.2: Sequence Allowing Slice</h2>

In object-oriented programming, a protocol is an informal interface defined only in documentation and not in code. For example, the sequence protocol in Python implies only the `__len__` and `__getitem__` methods.<br>
Implementation of the sequence protocol in the `Vector` class.

In [5]:
import math
import reprlib
from array import array


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['): -1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes(ord[self.typecode]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        return self._components[index]

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)

In [6]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [7]:
v1[0]

3.0

In [8]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

<h3>Method <code>__getitem__</code> with Consideration of Slices</h3>

In [9]:
import math
import operator
import reprlib
from array import array


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['): -1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes(ord[self.typecode]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            # Get an instance class and call the class to build
            # a new Vector instance by array slice
            cls = type(self)
            return cls(self._components[key])
        # Get index by key and return one specific item from _components
        index = operator.index(key)
        return self._components[index]

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)

In [10]:
v7 = Vector(range(7))
v7[-1]

6.0

In [11]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [12]:
v7[-1:]

Vector([6.0])

<h2><code>Vector</code> Attempt No.3: Access to Dynamic Attributes</h2>

In [13]:
import math
import operator
import reprlib
from array import array


class Vector:
    # Initialize __match_args__ so that pattern matching can be applied
    # to dynamic attributes supported by __getattr__
    __match_args__ = ('x', 'y', 'z', 't')

    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['): -1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes(ord[self.typecode]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    def __getattr__(self, name):
        cls = type(self)
        try:
            # Try to get the position of name in __match_args__
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        # If pos does not extend beyond the tuple
        # return the corresponding element
        if 0 <= pos < len(self._components):
            return self._components[pos]
        raise AttributeError(f"{cls.__name__!r} object has no attribute {name}")

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)

In [14]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

Access to element `x[0]` by name `v.x`

In [15]:
v.x

0.0

Assign a new value to `v.x` this should raise an exeption.

In [16]:
v.x = 10
v.x

10

In [17]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

To avoid a mismatch, change the logic of setting attributes in the `Vector` class.<br>
Implementation of the `__setattr__` method.

In [18]:
import math
import operator
import reprlib
from array import array


class Vector:
    __match_args__ = ('x', 'y', 'z', 't')

    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['): -1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes(ord[self.typecode]) + bytes(self._components)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        raise AttributeError(f"{cls.__name__!r} object has no attribute {name}")

    def __setattr__(self, name, value):
        cls = type(self)
        # Special handling of single-character attribute names
        if len(name) == 1:
            if name in cls.__match_args__:
                error = "readonly attribute {attr_name!r}"
            elif name.islower():
                error = "can't set attributes from 'a' to 'z' in {cls_name!r}"
            else:
                error = ''  # leave the error message empty
            if error:
                raise AttributeError(
                    error.format(cls_name=cls.__name__, attr_name=name)
                )
        # Default: call the __setattr__ method of the superclass
        # to get the standard behavior
        super().__setattr__(name, value)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)

In [19]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [20]:
try:
    v.x = 10
except AttributeError as e:
    print(e.__repr__())

AttributeError("readonly attribute 'x'")


In [21]:
try:
    v.w = 7
except AttributeError as e:
    print(e.__repr__())

AttributeError("can't set attributes from 'a' to 'z' in 'Vector'")


<h2><code>Vector</code> Attempt No.4: Hashing and Accelerating the Operator <code>==</code></h2>

In [22]:
import functools
import math
import operator
import reprlib
from array import array


class Vector:
    __match_args__ = ('x', 'y', 'z', 't')

    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['): -1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes(ord[self.typecode]) + bytes(self._components)

    def __eq__(self, other):
        """
        Checks if two objects are equal by comparing lengths and elements.

        If the lengths of `self` and `other` are not equal, returns `False`.
        If the lengths are equal, compares corresponding elements using `zip` and
        returns `False` if any pair of elements differ.
        Returns `True` if all pairs match.
        """

        return len(self) == len(other) and all(a == b for a, b in zip(self, other))

    def __hash__(self):
        # Create a genrating expression for delayed calculation
        # of the hash of each component
        hashes = (hash(x) for x in self._components)
        # Combine hash values using `operator.xor` (^) operator
        return functools.reduce(operator.xor, hashes, 0)  # initial value is 0

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        raise AttributeError(f"{cls.__name__!r} object has no attribute {name}")

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.__match_args__:
                error = "readonly attribute {attr_name!r}"
            elif name.islower():
                error = "can't set attributes from 'a' to 'z' in {cls_name!r}"
            else:
                error = ''  # leave the error message empty
            if error:
                raise AttributeError(
                    error.format(cls_name=cls.__name__, attr_name=name)
                )
        super().__setattr__(name, value)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)

In [23]:
v1 = Vector([3, 4])
v2 = Vector([3.1, 4.2])
v3 = Vector([3, 4, 5])
v6 = Vector(range(6))

In [24]:
hash(v1), hash(v2), hash(v3), hash(v6)

(7, 384307168202284039, 2, 1)

<h2><code>Vector</code> Attempt No.5: Formatting</h2>

In [25]:
import functools
import itertools
import math
import operator
import reprlib
from array import array


class Vector:
    __match_args__ = ('x', 'y', 'z', 't')

    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['): -1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes(ord[self.typecode]) + bytes(self._components)

    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))

    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        raise AttributeError(f"{cls.__name__!r} object has no attribute {name}")

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.__match_args__:
                error = "readonly attribute {attr_name!r}"
            elif name.islower():
                error = "can't set attributes from 'a' to 'z' in {cls_name!r}"
            else:
                error = ''  # leave the error message empty
            if error:
                raise AttributeError(
                    error.format(cls_name=cls.__name__, attr_name=name)
                )
        super().__setattr__(name, value)

    def angle(self, n):
        """
        Calculate one of the angular coordinates.
        """
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n - 1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        """
        Create a generating expression to calculate all angular coordinates on demand.
        """
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            # Use itertools.chain to generate a generating expression
            # that tries the modulus and angular coordinates of a vector
            coords = itertools.chain([abs(self)], self.angles())
            # Configure display of spherical coordinates in angle brackets
            outer_fmt = '<{}>'
        else:
            coords = self
            # Configure display of Cartesian coordinates in the round brackets
            outer_fmt = '({})'
        # Create a generating expression to format coordinates
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:].cast(typecode))
        return cls(memv)

Using `__format__` with Cartesian coordinates in 2D

In [26]:
v1 = Vector([3, 4])
format(v1)

'(3.0, 4.0)'

In [27]:
format(v1, '.2f')

'(3.00, 4.00)'

In [28]:
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

Using `__format__` with Cartesian coordinates in 3D and 7D

In [29]:
v3 = Vector([3, 4, 5])
format(v3)

'(3.0, 4.0, 5.0)'

In [30]:
format(Vector(range(7)))

'(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'

Using `__format__` with spherical coordinates in 2D, 3D, and 4D

In [31]:
format(Vector([1, 1]), 'h')

'<1.4142135623730951, 0.7853981633974483>'

In [32]:
format(Vector([1, 1]), '.3eh')

'<1.414e+00, 7.854e-01>'

In [33]:
format(Vector([1, 1]), '0.5fh')

'<1.41421, 0.78540>'

In [34]:
format(Vector([1, 1, 1]), 'h')

'<1.7320508075688772, 0.9553166181245093, 0.7853981633974483>'

In [35]:
format(Vector([2, 2, 2]), '.3eh')

'<3.464e+00, 9.553e-01, 7.854e-01>'

In [36]:
format(Vector([0, 0, 0]), '0.5fh')

'<0.00000, 0.00000, 0.00000>'

In [37]:
format(Vector([-1, -1, -1, -1]), 'h')

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [38]:
format(Vector([2, 2, 2, 2]), '.3eh')

'<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'

In [39]:
format(Vector([0, 1, 0, 0]), '0.5fh')

'<1.00000, 1.57080, 0.00000, 0.00000>'